<a href="https://colab.research.google.com/github/2pterons/training/blob/main/NLP/08)_BERT%EC%9D%98_%EB%AC%B8%EC%9E%A5_%EC%9E%84%EB%B2%A0%EB%94%A9(SBERT)%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_%ED%95%9C%EA%B5%AD%EC%96%B4_%EC%B1%97%EB%B4%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 08) BERT의 문장 임베딩(SBERT)을 이용한 한국어 챗봇

SBERT를 이용하여 문장 임베딩을 얻을 수 있는 패키지인 sentence_transformers를 사용하여 쉽고 간단하게 한국어 챗봇을 구현해봅시다. 실습에 앞서 sentence_transformers를 설치합니다.

In [ ]:
!pip install sentence_transformers

트랜스포머를 이용한 한국어 챗봇 실습에서 사용했던 데이터를 그대로 사용합니다.

In [ ]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
from sentence_transformers import SentenceTransformer

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
train_data.head()

('ChatBotData.csv', <http.client.HTTPMessage at 0x7f6866976110>)

문장 임베딩을 얻기 위해서 사전 훈련된 BERT를 로드합니다. 여기서는 한국어도 포함되어 학습된 다국어 모델을 로드합니다.

In [ ]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

모델의 이름은 'xlm-r-100langs-bert-base-nli-stsb-mean-tokens'인데 이름이 의미하는 바는 100가지 언어를 지원(한국어 포함)하는 다국어 BERT BASE 모델로 SNLI 데이터를 학습 후 STS-B 데이터로 학습되었으며, 문장 표현을 얻기 위해서는 평균 풀링(mean-tokens)을 사용했다는 의미입니다. 다시 말해서 NLI 데이터를 학습 후에 STS 데이터로 추가 파인 튜닝한 모델이라는 의미입니다.

SentenceTransformer로 로드할 수 있는 다양한 모델에 대한 리스트는 아래의 링크에서 확인 가능합니다.
해당 링크에 한국어 버전의 모델들 또한 공개되어 있으니 방문해보세요.

링크 : https://huggingface.co/models?library=sentence-transformers

데이터에서 모든 질문열. 즉, train_data['Q']에 대해서 문장 임베딩 값을 구한 후 embedding이라는 새로운 열에 저장합니다.

In [ ]:
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)

두 개의 벡터로부터 코사인 유사도를 구하는 함수 cos_sim를 정의합니다.

In [ ]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

return_answer 함수는 임의의 질문이 들어오면 해당 질문의 문장 임베딩 값과 챗봇 데이터의 임베딩 열. 즉, train_data['embedding']에 저장해둔 모든 질문 샘플들의 문장 임베딩 값들을 전부 비교하여 코사인 유사도 값이 가장 높은 질문 샘플을 찾아냅니다. 그리고 해당 질문 샘플과 짝이 되는 답변 샘플을 리턴합니다.

In [ ]:
def return_answer(question):
    embedding = model.encode(question)
    train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    return train_data.loc[train_data['score'].idxmax()]['A']

이제 챗봇을 테스트해봅시다.

In [ ]:
return_answer('결혼하고싶어')

'좋은 사람이랑 결혼할 수 있을 거예요.'

In [ ]:
return_answer('나랑 커피먹을래?')

In [ ]:
return_answer('반가워')

'저도 반가워요.'

In [ ]:
return_answer('사랑해')

'상대방에게 전해보세요.'

In [ ]:
return_answer('너는 누구니?')

'저는 위로봇입니다.'

In [ ]:
return_answer('너무 짜증나')

'짜증날 땐 짜장면'

In [ ]:
return_answer('화가납니다')

'화를 참는 연습을 해보세요.'

In [ ]:
return_answer('나랑 놀자')

'지금 그러고 있어요.'

In [ ]:
return_answer('나랑 게임하자')

'같이 놀아요.'

In [ ]:
return_answer('출근하기 싫어')

'직장 스트레스가 심한가봐요.'

In [ ]:
return_answer('여행가고싶다')

'이김에 떠나보세요.'

In [ ]:
return_answer('너 말 잘한다')

'그런 사람이 있으면 저 좀 소개시켜주세요.'

짧은 질문들이지만 대부분의 질문에서 그럴듯한 답변을 하는 모습을 확인할 수 있었습니다.

<hr/>
https://peltarion.com/knowledge-center/documentation/modeling-view/build-an-ai-model/blocks/xlm-r-encoder

한국어 버전 SBERT Colab link : https://colab.research.google.com/drive/1LiKIo493CLcXx9wcL68NOkW8h2CctBuW?usp=sharing